In [1]:
import os
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
train_ds = image_dataset_from_directory(
    'data/tiled_images/keras',
    batch_size=32,
    subset='training',
    seed=123,
    validation_split=0.3
).map(lambda x, y: (x/255, y))

val_ds = image_dataset_from_directory(
    'data/tiled_images/keras',
    batch_size=32,
    subset='validation',
    seed=123,
    validation_split=0.3
).map(lambda x, y: (x/255, y))

Found 378307 files belonging to 2 classes.
Using 264815 files for training.
Found 378307 files belonging to 2 classes.
Using 113492 files for validation.


In [3]:
# train_ds = image_dataset_from_directory(
#     'data/keras_subset',
#     batch_size=32,
#     subset='training',
#     seed=123,
#     validation_split=0.2
# ).map(lambda x, y: (x/255, y))

# val_ds = image_dataset_from_directory(
#     'data/keras_subset',
#     batch_size=32,
#     subset='validation',
#     seed=123,
#     validation_split=0.2
# ).map(lambda x, y: (x/255, y))

In [6]:
model1 = keras.models.Sequential([
    Conv2D(32, 3, activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(2),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(2),    
    Flatten(),
    Dense(1, activation='sigmoid')
])

es = EarlyStopping(monitor = 'val_loss', patience = 3)
cp = ModelCheckpoint(filepath='model1_checkpoint.keras', monitor='val_acc', save_best_only=True, mode='max', verbose=1)

model1.compile(loss='bce', optimizer='Adam', metrics=['acc'])

In [ ]:
history1 = model1.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[es, cp])

Epoch 1/20
8276/8276 [==============================] - ETA: 0s - loss: 0.5113 - acc: 0.7547
Epoch 1: val_acc improved from -inf to 0.80592, saving model to model1_checkpoint.keras
8276/8276 [==============================] - 5299s 640ms/step - loss: 0.5113 - acc: 0.7547 - val_loss: 0.4461 - val_acc: 0.8059
Epoch 2/20
3979/8276 [=============>................] - ETA: 43:26 - loss: 0.4518 - acc: 0.8012

In [ ]:
# The saved model can later be loaded using 
model1 = keras.models.load_model('model1_checkpoint.keras')

In [ ]:
model1.summary()

In [ ]:
# Check out the plot of loss vs epoch. How's our model?
plt.figure(figsize=(12, 6))
plt.plot(history.history1['loss'], c='navy', label='Training Loss')
plt.plot(history.history1['val_loss'], c='skyblue', label='Validation Loss')
plt.legend();